In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('C:/Users/jangmark/deeplm/data/gyungju.csv')

In [3]:
# dimension
df.shape

(12036, 11)

In [4]:
# 결측치
df.isnull().sum()

Unnamed: 0         0
리뷰              2293
별점                 0
리뷰자 평균 평점          0
리뷰자 누적 리뷰 개수       0
영업 시간             56
이미지url             0
가게명                0
가게 주소              0
메뉴               586
카카오 url            0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.isnull().sum()

Unnamed: 0      0
리뷰              0
별점              0
리뷰자 평균 평점       0
리뷰자 누적 리뷰 개수    0
영업 시간           0
이미지url          0
가게명             0
가게 주소           0
메뉴              0
카카오 url         0
dtype: int64

In [7]:
# 정규 표현식 함수 정의

import re

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result


In [8]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
  try:
    result_df = spell_checker.check(df['리뷰'][i])
    df['리뷰'][i] = result_df.as_dict()['checked']
  except:
    pass

100%|██████████| 9216/9216 [00:00<00:00, 658311.01it/s]


In [9]:
df['리뷰'][0]

'수성못 옆이라서 분위기는 좋음. 근데 내 돈주고 먹지는 않을듯'

In [10]:
apply_regular_expression(df['리뷰'][0])

'수성못 옆이라서 분위기는 좋음 근데 내 돈주고 먹지는 않을듯'

In [11]:
from konlpy.tag import Okt
from collections import Counter

In [12]:
okt = Okt()  # 명사 형태소 추출 함수
nouns = okt.nouns(apply_regular_expression(df['리뷰'][0]))



In [13]:
# 말뭉치 생성
corpus = "".join(df['리뷰'].tolist())

In [32]:
# 정규 표현식 적용
apply_regular_expression(corpus)

'수'

In [15]:
# 전체 말뭉치(corpus)에서 명사 형태소 추출
nouns = okt.nouns(apply_regular_expression(corpus))
print(nouns)


['수성', '못', '옆']


In [16]:
# 빈도 탐색
counter = Counter(nouns)

In [27]:
counter.most_common(10)

[('맛', 3092),
 ('가격', 900),
 ('음식', 853),
 ('곳', 781),
 ('직원', 702),
 ('집', 693),
 ('진짜', 668),
 ('때', 668),
 ('좀', 653),
 ('고기', 614)]

In [26]:
WORD_NUMS = wordFreq[wordFreq[1]>=500].shape[0]

print(f'분석에 사용할 단어 수 : {WORD_NUMS}')


[('가격', 900),
 ('음식', 853),
 ('직원', 702),
 ('진짜', 668),
 ('고기', 614),
 ('사람', 579),
 ('손님', 578),
 ('그냥', 578),
 ('맛집', 547),
 ('정도', 528)]

In [25]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(df['리뷰'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

TypeError: 'CountVectorizer' object is not subscriptable